### Part 0: Pre-Processing

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [14]:
file_list = drive.ListFile({'q': "title contains 'NLPFinal_uploadedAt235' and trashed=false"}).GetList()
# file_list = drive.ListFile({})
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: NLPFinal_uploadedAt235, id: 1E99YL8LvU-mLizrQFR2r3DuLwkUuI1lg


In [4]:
import pandas as pd 
import numpy as np
import re
import nltk
stemmer = nltk.stem.porter.PorterStemmer()
df = pd.read_csv('../tweets.csv')
df[0:10]
print('The number of tweets is ', len(df['text']))

FileNotFoundError: ignored

In [2]:
df.dropna(subset=["text"], inplace=True)

retweetedRussianTweets = df.retweeted_status_id.isin(df.tweet_id) & df.retweeted_status_id.notnull()
df2 = df.drop(df[retweetedRussianTweets].index)

df_unique_stemmed = df2.drop_duplicates('text')

r = re.compile(r'RT @\w*:')
retweet_series = df_unique_stemmed.text.str.contains(r)
rt_field_series = df_unique_stemmed.retweeted_status_id.notnull()

df_unique_stemmed[0:10]
# df['text'] = df["text"].apply(lambda x: [stemmer.stem(y) for y in x])
url_regex = re.compile(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)')
# username_regex = re.compile(r'RT @\w*')
username_regex = re.compile(r'@(?=\w)')
hashtag_regex = re.compile(r'#(?=\w)')

df_unique_stemmed.text = (df_unique_stemmed.text.str.replace(r,repl='')
    .str.replace(url_regex,repl='')
    .str.replace(username_regex, repl='')
    .str.replace(hashtag_regex, repl='')
    .str.replace('\n', repl='')
    .str.replace('\r', repl='')       
    .apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))
                         )

df_unique_stemmed.dropna(subset=["text"], inplace=True)
df_unique_stemmed[~(~retweet_series & ~rt_field_series)]['text'].to_csv('real_stemmed.txt',index=False)
df_unique_stemmed[~retweet_series & ~rt_field_series]['text'].to_csv('russian_stemmed.txt',index=False)


print('The number of unique stemmed tweets is ', len(df_unique_stemmed['text']))
print('The number of unique normal tweets is ', len(df_unique_stemmed[~(~retweet_series & ~rt_field_series)]['text']))
print('The number of unique Russian tweets is ', len(df_unique_stemmed[~retweet_series & ~rt_field_series]['text']))

NameError: ignored

In [0]:
df = pd.read_csv('./tweets.csv')

df.dropna(subset=["text"], inplace=True)

retweetedRussianTweets = df.retweeted_status_id.isin(df.tweet_id) & df.retweeted_status_id.notnull()
df2 = df.drop(df[retweetedRussianTweets].index)

df_unique = df2.drop_duplicates('text')

r = re.compile(r'RT @\w*:')
retweet_series = df_unique.text.str.contains(r)
rt_field_series = df_unique.retweeted_status_id.notnull()

df_unique[0:10]
# df['text'] = df["text"].apply(lambda x: [stemmer.stem(y) for y in x])
url_regex = re.compile(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)')
# username_regex = re.compile(r'RT @\w*')
username_regex = re.compile(r'@(?=\w)')
hashtag_regex = re.compile(r'#(?=\w)')

df_unique.text = (df_unique.text.str.replace(r,repl='')
    .str.replace(url_regex,repl='')
    .str.replace(username_regex, repl='')
    .str.replace(hashtag_regex, repl='')
    .str.replace('\n', repl='')
    .str.replace('\r', repl=''))

df_unique.dropna(subset=["text"], inplace=True)

df_unique[~(~retweet_series & ~rt_field_series)]['text'].to_csv('real.txt',index=False)
df_unique[~retweet_series & ~rt_field_series]['text'].to_csv('russian.txt',index=False)
# df_original["russian"] = 0

print('The number of unique unstemmed tweets is ', len(df_unique['text']))
print('The number of unique normal tweets is ', len(df_unique[~(~retweet_series & ~rt_field_series)]['text']))
print('The number of unique Russian tweets is ', len(df_unique[~retweet_series & ~rt_field_series]['text']))

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


The number of unique unstemmed tweets is  173905
The number of unique normal tweets is  121200
The number of unique Russian tweets is  52705


### Part 1: Russian Tweet Classifier

In [0]:
## Add a column bool_russian which is our target column for the classifier
df_unique_stemmed['bool_russian'] = (~retweet_series & ~rt_field_series).apply(lambda x: 1 if x else 0)
df_unique_stemmed[:10][['text','bool_russian']]

,text,bool_russian
0,thingsdonebymistak kiss aunti in the lip,1
1,theolderweget the more pessimist we are,0
2,readi To feel like A failure? joan Of arc wa ...,0
3,amen! blacklivesmatt,1
4,twitchy: chuck todd caught out there shill fo...,0
5,berniesand trump peopl should ralli togeth ag...,0
6,hillaryclinton the undecid voter on that stag...,0
7,todaycleveland 'no way',1
8,"nicktomawbr hi, nick! we'r hold a ""miner for t...",1
9,what. is. A. resolut my4wordnewyearsresolut,1


In [0]:
#dfLength = len(df_unique['text'])

## Here we will create out list of stopwords

from nltk.corpus import stopwords

stoplist = stopwords.words('english')
PS3StopWords = ["ever", "one", "do","does","make", "go", "us", "to", "get", 
                "about", "may", "s", ".", ",", "!", "i", "I", '\"', "?", ";", 
                "--", "--", "would", "could", "”",  "don’t", 
                "said", "can't", "didn't", "aren't", "I'm", "you're", "they're", "'s"]
stoplist.extend(PS3StopWords)
personalAdditions = ["it", "i'm", "|", "–", "-", "~", "you're", "thing", '"', "…", '…"', ""] # As determined by the top 100 most common tokens
stoplist.extend(personalAdditions)

## Here we use gensim corpora to create our dictionary of words
## This allows us to index our words to create our bag of words feature vector 

from gensim import corpora
tweets = [[word for word in tweet.lower().split() if word not in stoplist] for tweet in df_unique_stemmed['text']]
dictionary = corpora.Dictionary(tweets)

print(tweets[0:10],'\n')

## We remove stop words and words with frequency less than 20

cutoffnumber = 20

from six import iteritems
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
           if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq < cutoffnumber]
dictionary.filter_tokens(stop_ids + once_ids)  # remove stop words and words that appear only once
dictionary.compactify()  # remove gaps in id sequence after words that were removed
print(dictionary)


[['thingsdonebymistak', 'kiss', 'aunti', 'lip'], ['theolderweget', 'pessimist'], ['readi', 'feel', 'like', 'failure?', 'joan', 'arc', 'wa', 'onli', '19', 'wa', 'burn', 'stake'], ['amen!', 'blacklivesmatt'], ['twitchy:', 'chuck', 'todd', 'caught', 'shill', 'hillari', 'clintonth', 'post', 'busted:', 'adam', 'baldwi...', '#…'], ['berniesand', 'trump', 'peopl', 'ralli', 'togeth', 'establish', '💩-ing', 'choic', 'thefix'], ['hillaryclinton', 'undecid', 'voter', 'stage', 'wa', 'poll', 'trump', 'won.', 'cnn', 'biased.'], ['todaycleveland', "'no", "way'"], ['nicktomawbr', 'hi,', 'nick!', "we'r", 'hold', '"miner', 'trump"', 'ralli', 'tomorrow.', "you'r", 'interest', 'cover', 'it,', 'ple…'], ['what.', 'is.', 'a.', 'resolut', 'my4wordnewyearsresolut']] 

Dictionary(7363 unique tokens: ['kiss', 'lip', 'thingsdonebymistak', '19', 'burn']...)


In [0]:
## This following nested for loop creates the raw counts for the feature vectors

print(len(df_unique_stemmed['text'])) # Number of tweets
print(len(dictionary)) # Length of the feature vectors

featureVectors = np.zeros((len(df_unique_stemmed['text']),len(dictionary)),int)

for i in range(len(df_unique_stemmed['text'])) :
    for word in tweets[i]:
        if word in dictionary.token2id :
            word_id = dictionary.token2id[word]
            featureVectors[i][word_id] += 1
            
print(featureVectors[0]) ## Example feature vector

173905
7363
[1 1 1 ..., 0 0 0]


In [0]:
## Here we split our data into training set and test set

from sklearn.model_selection import train_test_split

featureVectors.shape[0]
targetvalues = df_unique_stemmed['bool_russian'].tolist()
featureVectors_train, featureVectors_test, target_train, target_test = train_test_split(featureVectors, targetvalues, random_state=0)


In [0]:
## Here we train our neural net with the training data

from sklearn.neural_network import MLPClassifier
layers = (10,5,2)
iters = 5
mlp = MLPClassifier(hidden_layer_sizes=layers,max_iter = iters)
mlp.fit(featureVectors_train,target_train)

/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=5, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [0]:
## Here we test out model and see its results

predictions = mlp.predict(featureVectors_test)

print(iters, ' iterations')
print(layers, 'layers')

from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(target_test,predictions))
print(classification_report(target_test,predictions))


5  iterations
(10, 5, 2) layers
[[26804  3528]
 [ 4712  8433]]
             precision    recall  f1-score   support

          0       0.85      0.88      0.87     30332
          1       0.71      0.64      0.67     13145

avg / total       0.81      0.81      0.81     43477



## Part 2: Topic Modelling

In [0]:
import gensim
from nltk.corpus import stopwords
import numpy as np
import scipy as sp
import re
from sklearn.cluster import KMeans
import nltk

In [0]:
## This code will split the texts into training data and test data

list_tweetstxt = df_unique_stemmed['text'].tolist()

from sklearn.model_selection import train_test_split

tweetstxt_train, tweetstxt_test = train_test_split(list_tweetstxt, random_state=0)

print(len(tweetstxt_train))
print(len(tweetstxt_test))

130428
43477


In [0]:
#### READ IN THE TWEETS

tweets = []     # original tweets
tweettoks = []  # list of lists of tokens by tweet
allTokens = []  # all of the tokens, used to determine most common tokens and thereby enhance the stoplist (see part 1 for implementation of this stoplist)
 
# Add process tweet text
for line in tweetstxt_train:
    line = line.rstrip()
    tweets.append(line)    
    line = re.sub(r"(^| )[0-9]+($| )", r" ", line)  # remove digits
    addMe = []
    for token in line.split():
        lowercaseToken = token.lower()
        if lowercaseToken not in stoplist:
            allTokens.append(lowercaseToken)
            addMe.append(lowercaseToken)
    tweettoks.append(addMe)

In [0]:
def printTweetTokenInfo():
    print("Length of tweettoks", len(tweettoks), '\n')

    print("Should be same as the number of tweets")
    print("Is same?", len(tweettoks)==len(tweets), '\n')

    print("EXAMPLE: First five tweets", tweets[0:5], '\n')
    
    print("EXAMPLE: First five tweets' token lists", tweettoks[0:5], '\n')
    
    print("Total number of tokens", len(allTokens), '\n')
    
    print("EXAMPLE: First 10 tokens (of all tweets)", allTokens[0:10], '\n')

In [0]:
#printTweetTokenInfo()  # Sanity check

## Part 2.2 Identify tweet topics for training data (using tweet token lists created above)


Identify topics in the tweets using word2vec word embeddings.

In [0]:
#### Create the word2vec model used to make tweet vectors in semantic space

bigmodel = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300-SLIM.bin", binary=True)

In [0]:
#### Using a word2vec model (taken from GoogleNews), read in the normalized vectors for each token, and sum the vectors to create a single tweet vector. Store that vector for future use / k-means clustering.

tweetvectors = []   # this list will contain one 300-dimensional vector per headline

for tweetTokens in tweettoks:
    totvec = np.zeros(300)
    for tok in tweetTokens:
        if tok.lower() in bigmodel:
            totvec = totvec + bigmodel[tok.lower()]
    tweetvectors.append(totvec)

#print(len(tweetvectors))      # Check that the number of vectors and tweets are the same
#print(len(tweets))            #     this number should be the same as the last
#print(len(tweetvectors[10]))  # Check to make sure this is a 300-Dimensional array

### Uncomment the following cell to build the topic model. 

This takes about 5 minutes. Otherwise, continue with saved model.

In [0]:
#### Use K-means clustering to sort our testing tweets into num_topics topics

#import pickle 
num_topics = 50

#kmtweets = KMeans(n_clusters=num_topics, random_state=0)
#tweetclusters = kmtweets.fit_predict(tweetvectors)

#filename = 'kmTweetsModel_50Topics.sav'
#pickle.dump(kmtweets, open(filename, 'wb'))

In [0]:
import pickle

# load the model from disk
loaded_kmtweets = pickle.load(open('kmTweetsModel_50Topics.sav', 'rb'))

In [0]:
##### Print out all the headlines that belong to one of the clusters. (UNCOMMENT CODE TO PRINT)

#desired_topic_cluster = 30
#
#for i in range(len(tweetclusters)):
#    if tweetclusters[i] == desired_topic_cluster:
#        print(tweets[i])

## Method to sort incoming tweets into the nearest cluster

This method will be used to sort test tweets into their nearest cluster, which will be included in our classifier as a feature from 1-num_clusters
This method takes in an array of tweets, and returns their intended cluster (based upon the above model)

(Our training data will use the cluster assigned to it-- contained in the list tweetclusters -- above as a fetaure in the final classifier.)

In [0]:
def assignClusters(tweets):
    
    ##### Separate each tweet by tokens
    
    toksByTweet = []

    for tweet in tweets:
        line = tweet.rstrip()
        line = re.sub(r"(^| )[0-9]+($| )", r" ", line)  # remove digits
        addMe = [token.lower() for token in line.split() if token.lower() not in stoplist]
        toksByTweet.append(addMe)
        
    ##### Turn each tweet's token list into a normalized (based upon the model, not the training tweets) vector
    
    vectors = []   # this list will contain one 300-dimensional vector per headline

    for toks in toksByTweet:
        totvec = np.zeros(300)
        for tok in toks:
            if tok.lower() in bigmodel:
                totvec = totvec + bigmodel[tok.lower()]
        vectors.append(totvec)
        
    
    #### Return the predicted topic

    return loaded_kmtweets.predict(vectors)

In [0]:
print("Let's check, does it correctly classify the first 50 training tweets?")        # Uncomment for a sanity check
print(assignClusters(tweets[0:50])==tweetclusters[0:50])

Let's check, does it correctly classify the first 50 training tweets?
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]


## Part 3: Tweet classifier with tweet topic included as a feature

In [0]:
featureVectors.shape[0]
featureVectors.shape[1]
featureVectors[0].shape[0]

new_feature_vectors = np.ndarray((featureVectors.shape[0], featureVectors.shape[1] + num_topics), int)

for i in range(featureVectors.shape[0]):
    topic_features = np.zeros(num_topics, int)
    assignClusters
    new_feature_vectors[i] = np.append(featureVectors[i],topic_features)